In [2]:
!pip install 'shimmy>=2.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 24.9 MB/s eta 0:00:0000:01
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.29.0
    Uninstalling gymnasium-0.29.0:
      Successfully uninstalled gymnasium-0.29.0
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 1.3.0
    Uninstalling Shimmy-1.3.0:
      Successfully uninstalled Shimmy-1.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.11 requires gymnasium==0.29.0, but you have gymnasium 1.1.1 which is incompatible.
stable-baselines3 2.1.0 requires gymnasium<0.30,>=0.28.1, but you have gymnasium 1.1.1 which is incompatible.


In [3]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 21.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [63]:
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from scipy.sparse import issparse, csr_matrix
from scipy.sparse.linalg import eigsh, splu
from scipy.sparse import identity
from scipy.sparse import random as sparse_random

# Hybrid Shift-and-Invert Power Method + RL
class ShiftInvertPowerRL:
    def __init__(self, A, sigma=0.0, shift_tol=1e-6):
        self.A = A
        self.sigma = sigma  # Shift value
        self.shift_tol = shift_tol
        self.dim = A.shape[0]
        self._precompute_shifted_matrix()

    def _precompute_shifted_matrix(self):
        """Precompute (A - σI)^-1 for sparse systems"""
        if issparse(self.A):
            I = identity(self.dim, format='csr')
            self.M = self.A - self.sigma * I
            self.M_solver = splu(self.M.tocsc())
        else:
            self.M = self.A - self.sigma * np.eye(self.dim)
            self.M_inv = np.linalg.inv(self.M)

    def solve_shifted_system(self, b):
        """Solve (A - σI)x = b"""
        if issparse(self.A):
            return self.M_solver.solve(b)
        else:
            return np.linalg.solve(self.M, b)

    def hybrid_power_iteration(self, num_iter=10):
        """Warm start using shift-and-invert power method"""
        x = np.random.randn(self.dim)
        x /= np.linalg.norm(x)

        for _ in range(num_iter):
            x = self.solve_shifted_system(x)
            x /= np.linalg.norm(x) + 1e-8
        return x

# RL Environment for Eigenvector Refinement
class ShiftInvertEnv(gym.Env):
    def __init__(self, A, sigma, target_eigenvalue, num_iter):
        super().__init__()
        self.solver = ShiftInvertPowerRL(A, sigma)
        self.target_eigenvalue = target_eigenvalue
        self.dim = A.shape[0]
        self.num_iter = num_iter # Number of iterations for hybrid power method
        # Observation: current vector + residual history
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                           shape=(self.dim + 3,), dtype=np.float32)

        # Action: vector adjustment directions
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.dim,), dtype=np.float32)

        # Initialize with hybrid power method
        self.state = self.solver.hybrid_power_iteration(num_iter=num_iter)
        self.residual_history = []

    def reset(self):
        self.state = self.solver.hybrid_power_iteration(num_iter=self.num_iter)
        self.residual_history = []
        return self._get_obs()

    def step(self, action):
        # RL-guided adjustment
        adjusted_vec = self.state + 0.1 * action
        adjusted_vec /= np.linalg.norm(adjusted_vec) + 1e-8

        # Power method step for stability
        refined_vec = self.solver.solve_shifted_system(adjusted_vec)
        refined_vec /= np.linalg.norm(refined_vec) + 1e-8

        # Calculate metrics
        residual = self.solver.A @ refined_vec - self.target_eigenvalue * refined_vec
        residual_norm = np.linalg.norm(residual)
        self.residual_history.append(residual_norm)

        # Reward shaping
        reward = -residual_norm - 0.1 * np.log(residual_norm + 1e-8)

        self.state = refined_vec
        done = residual_norm < self.solver.shift_tol

        return self._get_obs(), reward, done, {}

    def _get_obs(self):
        """Augment state with residual statistics"""
        residual_stats = [
            np.mean(self.residual_history[-10:]),
            np.min(self.residual_history[-10:]),
            np.std(self.residual_history[-10:])
        ] if self.residual_history else [0, 0, 0]
        return np.concatenate([self.state, residual_stats])

# Training Function
def train_hybrid_eigen_solver(A, sigma=0.0, total_timesteps=50000, num_iter=5):
    # Compute target eigenvalue and actual eigenvector
    eigvals, eigvecs = eigsh(A, k=1, sigma=sigma)
    target_eigenvalue = eigvals[0]
    actual_eigenvector = eigvecs[:, 0]

    # Initialize environment
    env = ShiftInvertEnv(A, sigma, target_eigenvalue, num_iter=num_iter)

    # RL Configuration
    policy_kwargs = dict(net_arch=[256, 256, 128])
    model = PPO(
        "MlpPolicy",
        env,
        learning_rate=1e-4,
        gamma=0.99,
        policy_kwargs=policy_kwargs,
        verbose=1
    )

    # Train
    model.learn(total_timesteps=total_timesteps)

    # Save model for inference
    model.save("hybrid_eigen_solver")

    # Evaluate
    obs = env.reset()
    best_vec = env.state
    best_residual = np.inf

    for _ in range(1000):
        action, _ = model.predict(obs)
        obs, _, _, _ = env.step(action)
        current_residual = np.linalg.norm(A @ env.state - target_eigenvalue * env.state)

        if current_residual < best_residual:
            best_residual = current_residual
            best_vec = env.state

    # Compute cosine distance
    v1 = best_vec / (np.linalg.norm(best_vec) + 1e-8)
    v2 = actual_eigenvector / (np.linalg.norm(actual_eigenvector) + 1e-8)
    cos_sim = np.dot(v1, v2)
    cos_sim = np.clip(cos_sim, -1.0, 1.0)  # Handle numerical errors
    cos_dist = 2 * np.arccos(abs(cos_sim)) / np.pi

    print(f"Training Final residual norm: {best_residual:.4e}")
    print(f"Training Eigenvector norm check: {np.linalg.norm(best_vec):.4f}")
    print(f"Training Cosine distance to actual eigenvector: {cos_dist:.6f}")

    return model, best_vec, best_residual, cos_dist, actual_eigenvector, target_eigenvalue

# Inference Function
def infer_eigenvector(model, A, sigma=0.0, num_iter=20):
    # Compute target eigenvalue and actual eigenvector
    eigvals, eigvecs = eigsh(A, k=1, sigma=sigma)
    target_eigenvalue = eigvals[0]
    actual_eigenvector = eigvecs[:, 0]

    # Initialize environment with new matrix
    env = ShiftInvertEnv(A, sigma, target_eigenvalue, num_iter)

    # Evaluate
    obs = env.reset()
    best_vec = env.state
    best_residual = np.inf

    for _ in range(1000):
        action, _ = model.predict(obs)
        obs, _, _, _ = env.step(action)
        current_residual = np.linalg.norm(A @ env.state - target_eigenvalue * env.state)

        if current_residual < best_residual:
            best_residual = current_residual
            best_vec = env.state

    # Compute cosine distance
    v1 = best_vec / (np.linalg.norm(best_vec) + 1e-8)
    v2 = actual_eigenvector / (np.linalg.norm(actual_eigenvector) + 1e-8)
    cos_sim = np.dot(v1, v2)
    cos_sim = np.clip(cos_sim, -1.0, 1.0)
    cos_dist = 2 * np.arccos(abs(cos_sim)) / np.pi

    print(f"Inference Final residual norm: {best_residual:.4e}")
    print(f"Inference Eigenvector norm check: {np.linalg.norm(best_vec):.4f}")
    print(f"Inference Cosine distance to actual eigenvector: {cos_dist:.6f}")

    return best_vec, best_residual, cos_dist, actual_eigenvector, target_eigenvalue


In [43]:
# Usage Example
# Generate training matrix
np.random.seed(42)
size = 1000
A_train = sparse_random(size, size, density=0.01, format='csr')
A_train = (A_train + A_train.T) * 0.5  # Make symmetric

# Train on single matrix
print("\nTraining on 1000x1000 matrix")
model, train_vec, train_residual, train_cos_dist, _, _ = train_hybrid_eigen_solver(A_train, sigma=0.5)




Training on 1000x1000 matrix
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 277  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 212        |
|    iterations           | 2          |
|    time_elapsed         | 19         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.04242738 |
|    clip_fraction        | 0.337      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.42e+03  |
|    explained_variance   | -0.00787   |
|    learning_rate        | 0.0001     |
|    loss                 | -0.0844    |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0437    |
|    std                  | 1          |
|    value_loss           | 43.1       |
----------------------------------------
-----------

TypeError: ShiftInvertEnv.__init__() missing 1 required positional argument: 'num_iter'

In [45]:
# Generate inference matrix
np.random.seed(43)  # Different seed for variability
A_infer = sparse_random(size, size, density=0.01, format='csr')
A_infer = (A_infer + A_infer.T) * 0.5  # Make symmetric

# Infer on new matrix
print("\nInferring on new 1000x1000 matrix")
infer_vec, infer_residual, infer_cos_dist, _, _ = infer_eigenvector(model, A_infer, sigma=0.5)


Inferring on new 1000x1000 matrix
Inference Final residual norm: 6.1266e-04
Inference Eigenvector norm check: 1.0000
Inference Cosine distance to actual eigenvector: 0.004250


# comparision with classical Shift & Invert

In [53]:
import numpy as np
from scipy.sparse.linalg import spsolve
from scipy.sparse import identity
from numpy.linalg import norm

def shift_invert_power_method(A, sigma, max_iter=100, tol=1e-8, verbose=False):
    """
    Applies the shift-and-invert power method to find the eigenvalue of A closest to sigma.

    Parameters:
    - A: scipy.sparse.csr_matrix, symmetric sparse matrix
    - sigma: float, shift parameter
    - max_iter: int, maximum number of iterations
    - tol: float, convergence tolerance
    - verbose: bool, whether to print convergence info

    Returns:
    - eigenvalue: float, estimated eigenvalue closest to sigma
    - eigenvector: numpy array, associated normalized eigenvector
    """

    n = A.shape[0]
    I = identity(n, format='csr')
    A_shifted = A - sigma * I

    x = np.random.rand(n)
    x /= norm(x)

    for i in range(max_iter):
        y = spsolve(A_shifted, x)
        y_norm = norm(y)
        x_new = y / y_norm

        if verbose:
            print(f"Iter {i+1}: Residual norm = {norm(x - x_new):.2e}")

        if norm(x - x_new) < tol:
            break

        x = x_new

    # Rayleigh quotient
    eigenvalue = x.T @ A @ x
    return eigenvalue, x


In [47]:
# Generate inference matrix
np.random.seed(43)  # Different seed for variability
A_infer2 = sparse_random(size, size, density=0.01, format='csr')
A_infer2 = (A_infer2 + A_infer2.T) * 0.5  # Make symmetric




Inferring on new 1000x1000 matrix
Inference Final residual norm: 6.0206e-04
Inference Eigenvector norm check: 1.0000
Inference Cosine distance to actual eigenvector: 0.006293


In [64]:
eigen_val, actual_eigenvector = shift_invert_power_method(A=A_infer2, sigma=0.5,max_iter=3)

In [54]:
eigvals, eigvecs = eigsh(A_infer2, k=1, sigma=0.5)
target_eigenvalue = eigvals[0]
real_eigenvector = eigvecs[:, 0]

In [66]:
v1 = infer_vec / (np.linalg.norm(infer_vec) + 1e-8)
v2 = actual_eigenvector / (np.linalg.norm(actual_eigenvector) + 1e-8)
cos_sim = np.dot(v1, v2)
cos_sim = np.clip(cos_sim, -1.0, 1.0)  # Handle numerical errors
cos_dist = 2 * np.arccos(abs(cos_sim)) / np.pi
print(cos_dist)

0.010249143312658344


In [67]:
# Infer on new matrix
print("\nInferring on new 1000x1000 matrix")
infer_vec, infer_residual, infer_cos_dist, _, _= infer_eigenvector(model, A_infer2, sigma=0.5, num_iter=3)


Inferring on new 1000x1000 matrix
Inference Final residual norm: 5.9916e-04
Inference Eigenvector norm check: 1.0000
Inference Cosine distance to actual eigenvector: 0.005523


## Improvement
- Classical power method 0.01025  
- Using Hybrid Method 0.005523  
- The cosine distance halves by using RL.
- This shows that the RL model takes us the last mile which is harder to achieve with standard power method which gives diminishing returns